###### _**PARAMETROS INICIAIS**_

In [1]:
#Importar Biblioteca 
from jira import JIRA

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 5, Finished, Available, Finished)

In [2]:
# Parametros 
varEmail = mssparkutils.credentials.getSecret('https://akv-cliente.vault.azure.net/', 'Email') #Alteração de E-mail e/ou Senha somente no Key vault
varAPI_TOKEN = mssparkutils.credentials.getSecret('https://akv-cliente.vault.azure.net/', 'API-token')

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 6, Finished, Available, Finished)

In [3]:
# Credenciais de acesso

email = varEmail 
api_token = varAPI_TOKEN 

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 7, Finished, Available, Finished)

In [4]:
# Conexão com o JIRA
jira = JIRA('https://cliente.atlassian.net/', basic_auth=(email, api_token))

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 8, Finished, Available, Finished)

###### _**EXTRAÇÃO DOS DADOS**_

**Obtenha todos os cards**

In [5]:
# Ler ref_projeto
table_ref_projeto = spark.read.format("delta").load("Tables/ref_projeto")

# Filtrar usa_template = 1
filtrar_usa_template = table_ref_projeto.filter(table_ref_projeto.usa_template == 1)

# Selecionar coluna chave
chave = filtrar_usa_template.select("chave").rdd.flatMap(lambda x: x).collect()

# lista all_issues
all_issues = []

# Buscar issues no Jira
for key in chave:
    issues = jira.search_issues("project = " + key)
    all_issues.extend(issues)

# Atribui valor
issues = all_issues


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 9, Finished, Available, Finished)

###### **FILTRAR DADOS - CONFORME O TYPE**

Filtrar registro por type

In [6]:
def filtrar_issues_por_tipo(issues):
    tipos = ['História', 'Atividade', 'Produto Técnico', 'Correção', 'Defeito', 'Epic']
    issues_filtradas = {tipo: list(filter(lambda issue: issue.fields.issuetype.name == tipo, issues)) for tipo in tipos}
    return issues_filtradas

# Chama a função para filtrar os issues
issues_filtradas = filtrar_issues_por_tipo(issues)

# Acessa cada tipo de issue filtrada
Type_Historia = issues_filtradas['História']
Type_Atividade = issues_filtradas['Atividade']
Type_ProdutoTecnico = issues_filtradas['Produto Técnico']
Type_Correcao = issues_filtradas['Correção']
Type_Defeito = issues_filtradas['Defeito']
Type_Epic = issues_filtradas['Epic']


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 10, Finished, Available, Finished)

###### **TRANSFORMAR DADOS ANTES NO LAKEHOUSE**

**Conversão do formato list para Dataframe**

Conversão de **Histórias** no formato lista para dataframe

In [7]:
# Converter para lista - Historia
issue_list_Historia = []

for issue in Type_Historia:
    issue_info_Historia = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tamanho_estimado": int(issue.fields.customfield_10484) if issue.fields.customfield_10484 is not None else 0,
        "tamanho_realizado": int(issue.fields.customfield_10485) if issue.fields.customfield_10485 is not None else 0,
        "valor_estimado": int(issue.fields.customfield_10486) if issue.fields.customfield_10486 is not None else 0,
        "valor_realizado": int(issue.fields.customfield_10487) if issue.fields.customfield_10487 is not None else 0,
        "finalidade": str(issue.fields.customfield_10493) if issue.fields.customfield_10493 is not None else "", 
        "versao": issue.fields.customfield_10483,
        "item_pai": issue.fields.parent.key if hasattr(issue.fields, 'parent') else None,
        "chave_projeto":issue.fields.project.key,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None
    }
    issue_list_Historia.append(issue_info_Historia)

# Converta a lista de dicionários em um DataFrame Spark 
issue_df_Historia = spark.createDataFrame(issue_list_Historia)


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 11, Finished, Available, Finished)

Conversão de **Atividade** no formato lista para dataframe

In [8]:
# Converter para lista - Atividade
issue_list_Atividade= []

for issue in Type_Atividade:
    issue_info_Atividade = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tipo_atividade_basico": issue.fields.customfield_10488.value if hasattr(issue.fields.customfield_10488, 'value') else None,
        "tipo_atividade_detalhe": issue.fields.customfield_10488.child.value if hasattr(issue.fields.customfield_10488, 'child') else None,
        "horas_estimadas": issue.fields.customfield_10491,
        "horas_realizadas": issue.fields.customfield_10492, 
        "inicio_trabalho": issue.fields.customfield_10497,
        "termino_trabalho": issue.fields.customfield_10498,
        "item_pai": issue.fields.parent.key if hasattr(issue.fields, 'parent') else None,
        "chave_projeto":issue.fields.project.key,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None
    }
    issue_list_Atividade.append(issue_info_Atividade)

# Converta a lista de dicionários em um DataFrame Spark 
issue_df_Atividade = spark.createDataFrame(issue_list_Atividade)

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 12, Finished, Available, Finished)

Conversão de **Produto Técnico** no formato lista para dataframe

In [9]:
issue_list_ProdutoTecnico = []

for issue in Type_ProdutoTecnico:
    customfield_10115 = issue.fields.customfield_10115
    sprint_name = None
    if customfield_10115 is not None and isinstance(customfield_10115, list) and len(customfield_10115) > 0:
        sprint_name = customfield_10115[0].name if hasattr(customfield_10115[0], 'name') else None

    issue_info_ProdutoTecnico = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tamanho_estimado": int(issue.fields.customfield_10484) if issue.fields.customfield_10484 is not None else 0,
        "tamanho_realizado": int(issue.fields.customfield_10485) if issue.fields.customfield_10485 is not None else 0,
        "finalidade": str(issue.fields.customfield_10493) if issue.fields.customfield_10493 is not None else "", 
        "versao": issue.fields.customfield_10483,
        "sprint": sprint_name, 
        "item_pai": issue.fields.parent.key if hasattr(issue.fields, 'parent') else None,
        "chave_projeto": issue.fields.project.key,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None
    }
    issue_list_ProdutoTecnico.append(issue_info_ProdutoTecnico)

# Converta a lista de dicionários em um DataFrame Spark
issue_df_ProdutoTecnico = spark.createDataFrame(issue_list_ProdutoTecnico)


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 13, Finished, Available, Finished)

Conversão de **Correção** no formato lista para dataframe

In [10]:
# Converter para lista - Correção
issue_list_Correcao = []

for issue in Type_Correcao:
    issue_info_Correcao = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tipo_atividade_basico": issue.fields.customfield_10488.value if hasattr(issue.fields.customfield_10488, 'value') else None,
        "tipo_atividade_detalhe": issue.fields.customfield_10488.child.value if hasattr(issue.fields.customfield_10488, 'child') else None, 
        "horas_estimadas": issue.fields.customfield_10491, 
        "horas_realizadas": issue.fields.customfield_10492, 
        "inicio_trabalho": issue.fields.customfield_10497,
        "termino_trabalho": issue.fields.customfield_10498,
        "item_pai": issue.fields.parent.key if hasattr(issue.fields, 'parent') else None,
        "chave_projeto": issue.fields.project.key,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None

    }
    issue_list_Correcao.append(issue_info_Correcao)

# Converta a lista de dicionários em um DataFrame Spark 
issue_df_Correcao = spark.createDataFrame(issue_list_Correcao)

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 14, Finished, Available, Finished)

Conversão de **Defeito** no formato lista para dataframe

In [11]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Defina o esquema explicitamente
schema = StructType([
    StructField("chave_jira", StringType(), True),
    StructField("titulo", StringType(), True),
    StructField("tamanho_estimado", IntegerType(), True),
    StructField("tamanho_realizado", IntegerType(), True),
    StructField("versao", StringType(), True),
    StructField("sprint", StringType(), True),
    StructField("item_pai", StringType(), True),
    StructField("chave_projeto", StringType(), True),
    StructField("nome_projeto", StringType(), True),
    StructField("tipo_produto", StringType(), True),
    StructField("status", StringType(), True)
])

# Converter para lista - Defeito
issue_list_Defeito = []

for issue in Type_Defeito:
    customfield_10115 = issue.fields.customfield_10115
    sprint_name = None
    if customfield_10115 and isinstance(customfield_10115, list) and customfield_10115:
        sprint_name = getattr(customfield_10115[0], 'name', None)

    issue_info_Defeito = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tamanho_estimado": int(issue.fields.customfield_10484) if issue.fields.customfield_10484 is not None else 0,
        "tamanho_realizado": int(issue.fields.customfield_10485) if issue.fields.customfield_10485 is not None else 0,
        "versao": issue.fields.customfield_10483,
        "sprint": sprint_name,
        "item_pai": issue.fields.parent.key if hasattr(issue.fields, 'parent') else None,
        "chave_projeto": issue.fields.project.key,
        "tipo_produto": issue.fields.customfield_10495.value if hasattr(issue.fields.customfield_10495, 'value') else None,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None
    }
    issue_list_Defeito.append(issue_info_Defeito)

# Converta a lista de dicionários em um DataFrame Spark usando o esquema definido
issue_df_Defeito = spark.createDataFrame(issue_list_Defeito, schema=schema)


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 15, Finished, Available, Finished)

Conversão de **Epic** no formato lista para dataframe

In [12]:
# Converter para lista - Epic
issue_list_Epic = []

for issue in Type_Epic:
    issue_info_Epic = {
        "chave_jira": issue.key,
        "titulo": issue.fields.summary,
        "tipo_produto": issue.fields.customfield_10495.value if hasattr(issue.fields.customfield_10495, 'value') else None,
        "chave_projeto":issue.fields.project.key,
        "nome_projeto": issue.fields.project.name,
        "status": issue.fields.status.name if hasattr(issue.fields.status, 'name') else None
    }
    issue_list_Epic.append(issue_info_Epic)

# Converta a lista de dicionários em um DataFrame Spark 
issue_df_Epic = spark.createDataFrame(issue_list_Epic, schema=schema)

StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 16, Finished, Available, Finished)

###### **_INSERÇÃO DOS DADOS EM FORMATO PARQUET_**

Inserção dos items no Lakehouse

In [13]:
def salvar_dataframes(dataframes, nomes):
    for df, nome in zip(dataframes, nomes):
        df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save(f"Tables/{nome}")

# Lista de DataFrames
dataframes = [
    issue_df_Historia,
    issue_df_Atividade,
    issue_df_ProdutoTecnico,
    issue_df_Correcao,
    issue_df_Defeito,
    issue_df_Epic
]

# Lista de nomes dos arquivos
nomes = [
    "jts_historia",
    "jts_atividade",
    "jts_produto_tecnico",
    "jts_correcao",
    "jts_defeito",
    "jts_epico"
]

# Salvar os DataFrames
salvar_dataframes(dataframes, nomes)


StatementMeta(, e8864c26-6f11-4b3f-b6a0-a0fdd14fa749, 17, Finished, Available, Finished)